# Accessing images from the loc.gov JSON API for image analysis

The digital collections' images available from the [Library of Congress website](https://www.loc.gov) are an amazing resource for study and analysis using digital methods. This notebook shows how you can use the loc.gov JSON API and Python to access sets of images.

More information about API is at [About the loc.gov JSON API](https://libraryofcongress.github.io/data-exploration/). For more example code in Jupyter notebooks, check out [LC for Robots](https://labs.loc.gov/lc-for-robots). 

This tutorial contains several stages:

1. [Context](#Context)
2. [Identifying items](#Identifying-items)
3. [Calling the API and retrieving image URLs](#Calling-the-API-and-retrieving-image-URLs)
4. [Accessing images](#Accessing-images)
5. [Conclusion](#Conclusion)
   
## Version
---
Version: 2

Last Run: July 14, 2025 (Python 3.12)


---
Author Information:


Written by Laura Wrubel, Visiting Scholar 2018

Edited by Sabrina Templeton, Junior Fellow 2025

---

## Prerequisites
None. However, note that running this notebook will download files onto your machine. 
 

## Context

### Rights and access
Rights and restrictions, including copyright, affect how you can use images, particularly if you want to publish, display, or otherwise distribute them. You can read more in [About Copyright and the Collections](https://www.loc.gov/legal/). This notebook uses images from the Prints & Photographs Division (P&P), and you can read more about restrictions guidance for these materials at [Copyright and Other Restrictions That Apply to Publication/Distribution of Images:
Assessing the Risk of Using a P&P Image](http://www.loc.gov/rr/print/195_copr.html). There is also information about rights specific to the collection and item in several fields in the API response. See the end of this notebook for relevant fields.

### Image sizes and information about collections
In many cases, the images from the Prints & Photographs Division available via the API are a small thumbnail (150px on one side). Consider whether this is sufficient for your research methods and needs. If you need to work with higher resolution images than are available via the API, you'll need to consult with staff who work with the collection(s) to determine whether those are available for use on-site at the Library of Congress and what restrictions apply. Here are some examples of images that are 150px on one side: 

![https://www.loc.gov/item/2017691815/](https://cdn.loc.gov/service/pnp/fsa/8b02000/8b02800/8b02895_150px.jpg)
![https://www.loc.gov/item/2011645392/](https://cdn.loc.gov/service/pnp/ppmsca/31200/31267_150px.jpg)

It can also be helpful to discuss with staff your intended analysis, as factors such as the sources of metadata and how items were digitized may affect your findings. The [Ask a Librarian](http://www.loc.gov/rr/askalib/) service can help you get in touch with staff who work with the collections.

### More APIs for working with images
Outside of the Prints & Photographs Division images, most of the Library of Congress images are available from an [IIIF API](https://www.loc.gov/apis/micro-services/image-services/), including for example those of newspapers in the [Chronicling America](https://chroniclingamerica.loc.gov/) collection, [images from the Mansucript Division](https://www.loc.gov/search/?fa=online-format:image%7Cpartof:manuscript+division), and [images of maps](https://www.loc.gov/maps/?fa=online-format:image). The IIIF API allows you to zoom, rotate, resize, and work with images in more complex ways. For a brief intro to using the IIIF API, see [this Jupyter notebook on IIIF](https://github.com/LibraryOfCongress/data-exploration/blob/master/loc.gov%20IIIF%20API/IIIF.ipynb). 
 

## Identifying items 

So let's get started accessing images! If you haven't already, browse or search the [Library of Congress Digital Collections](https://www.loc.gov/collections) website to see what is available and refine your search parameters. The website lets you search over 500 curated collections by keyword, format, topic, and division. While the API documentation has [detailed information on parameters](https://www.loc.gov/apis/json-and-yaml/) you can use in your search, searching the collections often gives you good context that is useful in understanding the context of an image within its collection.

Once you've found a search that targets the items that interest you, **copy the URL**. That will be the **base URL** for your API request. 

For example: 

``https://www.loc.gov/collections/baseball-cards/``

``https://www.loc.gov/photos/?q=bridges&dates=1800/1899``

### Calling the API and retrieving image URLs

The function below will call the API, adding the following parameters:
* ``fo=json`` to get JSON format in response
* ``c=100`` a count of 100 results in each response, rather than the default 25
* ``at=results,pagination`` provide only the ``results`` and ``pagination`` parts of the response. The API response is otherwise very long with information
we don't need.


Each of the results in the `results` field will have a field called ``image_url``: 

``
"image_url": [
"//cdn.loc.gov/service/pnp/cph/3f00000/3f05000/3f05300/3f05332_150px.jpg",
"//cdn.loc.gov/service/pnp/cph/3f00000/3f05000/3f05300/3f05332_150px.jpg#h=150&w=100",
"//cdn.loc.gov/service/pnp/cph/3f00000/3f05000/3f05300/3f05332t.gif#h=150&w=100",
"//cdn.loc.gov/service/pnp/cph/3f00000/3f05000/3f05300/3f05332r.jpg#h=640&w=425",
"//cdn.loc.gov/service/pnp/cph/3f00000/3f05000/3f05300/3f05332v.jpg#h=1024&w=680"
],
``

For images from the Prints & Photographs Division, the last one listed is usually the largest of the image files publicly available. Below, the function retrieves the last item in the ``image_url`` array. 

In [1]:
import requests

def get_image_urls(url, items=[]):
    '''
    Retrieves the image URLs for items that have public URLs available. 
    Skips over items that are for the colletion as a whole or web pages about the collection.
    Handles pagination. 

    Args: 
        url (str): The URL to request a collection.
        items (list, optional): The list that fetched item URLs will get added to.

    Returns:
        list: The item URLS from the collection.  
    '''
    
    # request pages of 100 results at a time
    params = {"fo": "json", "c": 100, "at": "results,pagination"} 
    call = requests.get(url, params=params)
    data = call.json()
    results = data['results']
    for result in results:
        # don't try to get images from the collection-level result
        if "collection" not in result.get("original_format") and "web page" not in result.get("original_format"):
            # take the last URL listed in the image_url array
            if result.get("image_url"):
                item = result.get("image_url")[-1]
                items.append(item)
    if data["pagination"]["next"] is not None: # make sure we haven't hit the end of the pages
        next_url = data["pagination"]["next"]
        print("getting next page: {0}".format(next_url))
        time.sleep(3) # timer to avoid API rate limits
        get_image_urls(next_url, items)
        
    return items


The Library of Congress has a digitized collection of Baseball Cards collection from the late 19th and eary 20th century. Here's an example:

!["https://www.loc.gov/collections/static/baseball-cards/images/Ward0006.jpg"](https://www.loc.gov/collections/static/baseball-cards/images/Ward0006.jpg)

Let's retrieve URLs for images in the Baseball Cards collection. The base URL below will filter the results to those from the year 1888, allowing us to work with a more manageable number of files for the purpose of this notebook:

In [2]:
image_urls = get_image_urls("https://www.loc.gov/collections/baseball-cards/?dates=1888", items=[]) # remove the date parameter here to download the entire collection

How many URLs were retrieved?

In [3]:
len(image_urls)

58

As mentioned above, we filtered by date to receive a more manageable number of images for the purposes of this tutorial, as the total number of images in the Baseball Cards collection is over 2000. You can easily remove the date parameter above if you want to download the entirety of the collection.

Here are the URLs retrieved from the ``image_url`` field in the first five results in the API response. As you can see by the way the files have been named, these have one dimension being 1024 pixels. 

In [4]:
image_urls[:5]

['https://tile.loc.gov/storage-services/service/pnp/bbc/0300/0380/0382fv.jpg#h=1024&w=572',
 'https://tile.loc.gov/storage-services/service/pnp/bbc/0500/0530/0536fv.jpg#h=1024&w=561',
 'https://tile.loc.gov/storage-services/service/pnp/bbc/0500/0530/0537fv.jpg#h=1024&w=569',
 'https://tile.loc.gov/storage-services/service/pnp/bbc/0500/0550/0550fv.jpg#h=1024&w=554',
 'https://tile.loc.gov/storage-services/service/pnp/bbc/0500/0550/0551fv.jpg#h=1024&w=554']

You could pass this list of URLs to a tool like wget to collect the images. A helpful tutorial is [Automated Downloading with wget](https://programminghistorian.org/lessons/automated-downloading-with-wget). 

### Rights and restrictions data in the API

The API response also contains data about rights and restrictions, just as seen on the loc.gov website. The field may vary by collection, but can usually be found in `rights` and possibly also `rights_information` and `rights_advisory`. The `rights` field typically contains the rights statement from the collection level. For our collection, Baseball Cards, all three fields are present. Other collections may return errors, if `rights_information` and `rights_advisory` aren't used. The `access restricted` field is also a True/False boolean field used for toggling various rights restrictiosn and should not be taken as a comprehensive statement of the absense of all restrictions of any sort. 

To make an API request for a single item, use the URL in the ``id`` field of the results section and add ``fo=json``. 

We'll request item information for a specific item in the Works Progress Administration Posters collection and look at the rights associated with it: 

In [5]:
import json

# request JSON for a single item
r = requests.get("https://www.loc.gov/item/98513999/?fo=json")
r_data = r.json()

**rights_information**

In [6]:
print(json.dumps(r_data["item"]["rights_information"], indent=2))

"No known restrictions on publication."


**rights_advisory**

In [7]:
print(json.dumps(r_data["item"]["rights_advisory"], indent=2))

"No known restrictions on publication."


**rights**

In [8]:
print(json.dumps(r_data["item"]["rights"], indent=2))

[
  "<p>The Library of Congress does not&nbsp;own rights to material  in its collections. Therefore, it does not license or charge permission fees  for use of such material and cannot grant or deny permission to publish or  otherwise distribute the material. </p>\n<p>Ultimately, it is the researcher's obligation to assess copyright or other use restrictions and  obtain permission from third parties when necessary before publishing or  otherwise distributing materials found in the Library's collections. </p>\n<p>For information about reproducing,  publishing, and citing material from this collection, as well as access to the  original items, see: <a href=\"https://www.loc.gov/rr/print/res/217_wpa.html\">Work Projects Administration Posters - Rights and Restrictions Information</a> </p>\n\n"
]


**access_restricted**

If access is restricted, this field will be true. 

In [9]:
print(json.dumps(r_data["item"]["access_restricted"], indent=2))

false


## Accessing images

Now that you have the URLs for images, you can access them as part of your image analysis or download them. Just be sure to check out the rights and restrictions information about how you can distribute or display them.

The code below will acccess and save the images located the URLs you saved earlier. It saves them to a directory, so if you haven't already created a directory where you want to save the files, do that now. 

In [10]:
import os

In [11]:
os.mkdir("images")

In [12]:
def get_image_files(image_urls_list, path):
    '''
    Takes as input a list of URLs for loc.gov item pages and 
    a path to a directory in which to save image files, e.g. "data". 

    Args: 
        images_urls_list (list): The list of urls for which fetch the image files.
        path (str): The path to a directory in which image files will be saved. 

    Returns:
        None
    '''
 
    for count, url in enumerate(image_urls_list):
        if count % 10 == 0: # if you are getting 100s of images, change the 10 to 100 here to print more infrequently
            print("at item {0}".format(count))
        try:
            #filename = create a filename based on the last part of the URL.
            filename = url.split('/')[-1]
            filename = os.path.join(path, filename)
            # request the image and write to path
            image_response = requests.get(url, stream=True)
           
            with open(filename, 'wb') as fd:
                for chunk in image_response.iter_content(chunk_size=100000):
                    fd.write(chunk)
        except ConnectionError as e:
            print(e)

In [13]:
get_image_files(image_urls, "images")

at item 0
at item 10
at item 20
at item 30
at item 40
at item 50


Let's see what we downloaded.

In [14]:
os.listdir("images")

['0382fv.jpg#h=1024&w=572',
 '0536fv.jpg#h=1024&w=561',
 '0537fv.jpg#h=1024&w=569',
 '0550fv.jpg#h=1024&w=554',
 '0551fv.jpg#h=1024&w=554',
 '0552fv.jpg#h=1024&w=563',
 '0553fv.jpg#h=1024&w=547',
 '0554fv.jpg#h=1024&w=579',
 '0555fv.jpg#h=1024&w=595',
 '0556fv.jpg#h=1024&w=582',
 '0557fv.jpg#h=1024&w=587',
 '0558fv.jpg#h=1024&w=598',
 '0559fv.jpg#h=1024&w=588',
 '0560fv.jpg#h=1024&w=583',
 '0561fv.jpg#h=1024&w=547',
 '0562fv.jpg#h=1024&w=551',
 '0563fv.jpg#h=1024&w=567',
 '0564fv.jpg#h=1024&w=580',
 '0565fv.jpg#h=1024&w=544',
 '0566fv.jpg#h=1024&w=559',
 '0567fv.jpg#h=1024&w=540',
 '0568fv.jpg#h=1024&w=554',
 '0569fv.jpg#h=1024&w=660',
 '0570fv.jpg#h=1024&w=664',
 '0571fv.jpg#h=1024&w=666',
 '0572fv.jpg#h=1024&w=669',
 '0573fv.jpg#h=1024&w=679',
 '0574fv.jpg#h=1024&w=672',
 '0575fv.jpg#h=1024&w=657',
 '0576fv.jpg#h=677&w=1024',
 '0577fv.jpg#h=1024&w=671',
 '0578fv.jpg#h=1024&w=656',
 '0579fv.jpg#h=1024&w=661',
 '0580fv.jpg#h=1024&w=674',
 '0581fv.jpg#h=1024&w=661',
 '0582fv.jpg#h=1024&

This method works for situations where we you need a batch of images from one collection. However, if your images are from more than one collection, you might end up with files with the same name and end up overwriting them. And either way, these filenames don't tell you anything about the item. You might need to look up further metadata. So, here's an alternative approach that renames the file with the identifier used on the loc.gov website. We'll first re-fetch the image URL for each item and download the file, renaming it using the identifier. 

In [15]:
from urllib.parse import urlparse

def get_and_save_images(results_url, path):
    '''
    Takes as input the url for the collection or results set
    e.g. https://www.loc.gov/collections/baseball-cards
    and a list of items (used for pagination)

    Args:
        results_url (str): The url for the collection.
        path (str): The path in which images will be saved.
    '''
    params = {"fo": "json", "c": 100, "at": "results,pagination", "dates": "1888"} # remove the date parameter here to download the entire collection
    call = requests.get(results_url, params=params)
    data = call.json()
    results = data['results']
    for result in results:
        # don't try to get images from the collection-level result or web page results
        if "collection" not in result.get("original_format") and "web page" not in result.get("original_format"):
            if result.get("image_url"):
                image = result.get("image_url")[-1]
                # create a filename that's the identifier portion of the item URL
                identifier = urlparse(result["id"])[2].rstrip('/')
                identifier = identifier.split('/')[-1]
                filename = "{0}.jpg".format(identifier)
                filename = os.path.join(path, filename)
            
                # request the image and write to path
                image_response = requests.get(image, stream=True)
                with open(filename, 'wb') as fd:
                    for chunk in image_response.iter_content(chunk_size=100000):
                        fd.write(chunk)
    
    if data["pagination"]["next"] is not None: # make sure we haven't hit the end of the pages
        next_url = data["pagination"]["next"]
        print("getting next page: {0}".format(next_url))
        time.sleep(3) # timer to avoid API rate limits
        get_and_save_images(next_url, path)

In [16]:
os.mkdir("images-named")

In [17]:
get_and_save_images("https://www.loc.gov/collections/baseball-cards/", "images-named")

Let's take a quick look at the path for the directory to confirm that the images were downloaded. 

In [18]:
os.listdir("images-named")

['2007680728.jpg',
 '2007680729.jpg',
 '2007680730.jpg',
 '2007680731.jpg',
 '2007680732.jpg',
 '2007680733.jpg',
 '2007680734.jpg',
 '2007680735.jpg',
 '2007680736.jpg',
 '2007680737.jpg',
 '2007680738.jpg',
 '2007680739.jpg',
 '2007680740.jpg',
 '2007680741.jpg',
 '2007680742.jpg',
 '2007680743.jpg',
 '2007680744.jpg',
 '2007680745.jpg',
 '2007680746.jpg',
 '2007680747.jpg',
 '2007680748.jpg',
 '2007680749.jpg',
 '2007683700.jpg',
 '2007683701.jpg',
 '2007683702.jpg',
 '2007683703.jpg',
 '2007683704.jpg',
 '2007683705.jpg',
 '2007683706.jpg',
 '2007683750.jpg',
 '2007683751.jpg',
 '2007683752.jpg',
 '2007683753.jpg',
 '2007683754.jpg',
 '2007683755.jpg',
 '2007683756.jpg',
 '2007683757.jpg',
 '2007683758.jpg',
 '2007683759.jpg',
 '2007683760.jpg',
 '2007683761.jpg',
 '2007683762.jpg',
 '2007683763.jpg',
 '2007683764.jpg',
 '2007683765.jpg',
 '2007683766.jpg',
 '2007683767.jpg',
 '2007683768.jpg',
 '2007683769.jpg',
 '2007683770.jpg',
 '2007683771.jpg',
 '2007683772.jpg',
 '2007683773

### Connecting the image file to the metadata
The filename the code creates is the item's identifier, so you reconstruct a URL for the item's metadata. For example, to examine at the metadata for the first item in the list, 2007680728.jpg, you can add ``https://www.loc.gov/item/`` before the identifier. 

``https://www.loc.gov/item/2007680728``

You can also request the metadata in JSON format by adding ``?fo=json&at=item`` at the end. 

In [19]:
r = requests.get("https://www.loc.gov/item/2007680728/?fo=json")
r_data = r.json()
print(json.dumps(r_data["item"], indent=2))

{
  "_version_": 1730886823385759746,
  "access_advisory": "Restricted access: Materials in this collection are extremely fragile and cannot be served.",
  "access_restricted": false,
  "aka": [
    "http://www.loc.gov/pictures/collection/bbc/item/2007680728/",
    "https://hdl.loc.gov/loc.pnp/bbc.0550b",
    "http://www.loc.gov/item/2007680728/",
    "http://www.loc.gov/pictures/item/2007680728/",
    "https://hdl.loc.gov/loc.pnp/bbc.0550f",
    "http://www.loc.gov/resource/bbc.0550f/",
    "http://www.loc.gov/resource/bbc.0550b/",
    "http://lccn.loc.gov/2007680728"
  ],
  "call_number": "LOT 13163-06, no. 1 [P&P]",
  "campaigns": [],
  "contributor_names": [
    "Allen & Ginter, sponsor"
  ],
  "contributors": [
    {
      "allen & ginter": "https://www.loc.gov/search/?fa=contributor:allen+%26+ginter&fo=json"
    }
  ],
  "control_number": "",
  "created": "2016-04-21 00:37:59",
  "created_published": [
    "1888."
  ],
  "created_published_date": "1888.",
  "date": "1888-01-01",


## Conclusion

Access to batches of images opens up the door to new forms of analysis, research, and experimentation. 

We've shown where you can find information about rights and restrictions on use of images, how to find the image URLs in the API response, and how to save images if your analysis requires having the files on disk. 

Best of luck with your searching and let us know how your project goes!